In [ ]:
!pip install bbox-visualizer

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from sklearn import model_selection

import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import bbox_visualizer as bbv

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from glob import glob
from skimage import exposure

import torch
# Neural networks can be constructed using the torch.nn package.
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import torchvision
#from torchvision.models.detection import retinanet_resnet50_fpn
#from torchvision.models.detection.retinanet import RetinaNet

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import warnings

warnings.filterwarnings('ignore')

In [ ]:
BASE_DIR = '../input/vinbigdata-chest-xray-abnormalities-detection/'
WEIGHTS_FILE = './'

In [ ]:
train = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))
train.head()

In [ ]:
train = train[train.class_name!='No finding'].reset_index(drop=True)


In [ ]:
train_df, valid_df = train_test_split(train, test_size=0.33, random_state=42)


In [ ]:
class LungsAnnotationDataset(Dataset):
    def __init__(self,dataframe, image_dir, transforms=None):
        super().__init__()
        
        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        #self.labels = torch.nn.functional.one_hot(torch.tensor(dataframe.class_id))
        self.transforms = transforms
    
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]
        
        dcm_data = pydicom.read_file(f'{self.image_dir}/{image_id}.dicom')
        image = apply_voi_lut(dcm_data.pixel_array, dcm_data)
        # depending on this value, X-ray may look inverted - fix that:
        if dcm_data.PhotometricInterpretation == "MONOCHROME1":
            image = np.amax(image) - image
            
        #intercept = dcm_data.RescaleIntercept if "RescaleIntercept" in dcm_data else 0.0
        #slope = dcm_data.RescaleSlope if "RescaleSlope" in dcm_data else 1.0
        
        #if slope != 1:
        #    image = slope * image.astype(np.float64)
        #    image = image.astype(np.int16)
            
        #image += np.int16(intercept)
        image = np.stack([image, image, image])
        image = image - np.min(image)

        image = image / image.max()
        #image = image * 255.0
        #image = image.astype('float32')
        image = exposure.equalize_hist(image) #Normalization of X-ray images
        image = image.astype('float32')

        image = image.transpose(1,2,0)
        
        boxes = records[['x_min','y_min','x_max','y_max']].values
        #boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        #boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        
        labels = records.class_id.values + 1
        class_name = records.class_name.values
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = torch.tensor(boxes)
        target['labels'] = torch.tensor(labels)
        #target['class'] = class_name
        #target['image_id'] = torch.tensor([index])
        target['area'] = torch.tensor(area)
        target['iscrowd'] = torch.tensor(iscrowd)
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.tensor(sample['bboxes'])
            

        return image, target
    
    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
# Albumentations
def get_train_transform():
    return A.Compose([
        ToTensorV2(),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

DIR_TRAIN = os.path.join(BASE_DIR, "train")
train_dataset = LungsAnnotationDataset(train_df, DIR_TRAIN,get_train_transform())
valid_dataset = LungsAnnotationDataset(valid_df, DIR_TRAIN,get_valid_transform())

In [ ]:
train_data_loader = DataLoader(
    train_dataset,
    batch_size=6,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=6,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
images, targets = next(iter(train_data_loader))
images = list(image.to(device) for image in images)

targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [ ]:
print (targets)

In [ ]:

num_classes = 15 

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

print(model)

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load('../input/train-draft-folding-v0/model.pth', map_location=device))
model.train()

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0005, momentum=0.9, weight_decay=0.0005)
#torch.optim.Adam(params, lr = 1e-3)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
lr_scheduler = None

num_epochs = 4

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
def train_model(train_dataset, fold):
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=2,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
    )
    loss_hist = Averager()
    itr = 1
    for epoch in range(num_epochs):
        loss_hist.reset()
        model.train()
        for images, targets in train_data_loader:
            optimizer.zero_grad()
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            #losses = sum(loss for loss in loss_dict.values())
            loss_classifier, loss_box_reg, loss_objectness, loss_rpn_box_reg = loss_dict.values()
            losses = sum([loss_objectness, 
                         10*loss_classifier, 
                         10*loss_rpn_box_reg, 
                         (0.5*loss_box_reg**2)
                         ]
                        )
            loss_value = losses.item()
            loss_hist.send(loss_value)

            losses.backward()
            optimizer.step()
            if itr%100==0:
                print(f"Fold #{fold} Epoch #{epoch+1} Iteration #{itr} loss: {loss_hist.value}")
            
            itr += 1
            del loss_dict, loss_classifier, loss_box_reg, loss_objectness, loss_rpn_box_reg,loss_value
        itr=1    
        # update the learning rate
        if lr_scheduler is not None:
            lr_scheduler.step()

        print(f"Fold #{fold} Epoch #{epoch+1} loss: {loss_hist.value}")
        print("Saving Epoch's state...")
        torch.save(model.state_dict(), f"model_state.pth")

In [ ]:
k=1
df = train.sample(frac=1).reset_index(drop=True)
y = train.class_id.values
## GroupK-Fold Splitting
kfold = model_selection.GroupKFold(n_splits=5)


for train_index, val_index in kfold.split(df, y,groups=df.image_id.values):
    
    train_dataset = LungsAnnotationDataset(df.loc[val_index], DIR_TRAIN,get_train_transform())
    train_model(train_dataset, k)
    if k==5:
        valid_dataset = LungsAnnotationDataset(df.loc[train_index], DIR_TRAIN,get_train_transform())
        val_data_loader = DataLoader(
                                valid_dataset,
                                batch_size=6,
                                shuffle=False,
                                num_workers=4,
                                collate_fn=collate_fn
                            )
        
    k+=1

In [ ]:
torch.save(model.state_dict(), "model.pth")
from IPython.display import FileLink
FileLink(r'model.pth')